In [1]:
import os

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd


In [2]:
import matplotlib.pyplot as plt
import matplotlib
from matplotlib.pyplot import figure
import matplotlib as mpl
     
plt.rcParams.update({
    'font.size': 10,
    'text.usetex': True,
    'text.latex.preamble': r'\usepackage{amsfonts}\usepackage{bm}'
})
mpl.rc('font',family='Times New Roman')
matplotlib.rcParams['pdf.fonttype'] = 42
matplotlib.rcParams['ps.fonttype'] = 42
plt.rcParams['figure.figsize'] = (16, 9)
fontsize = 30
matplotlib.rcParams.update({'font.size': fontsize})

In [3]:
# Get all file names in current folder
files = os.listdir(".")

In [4]:
# Concatenate all dataframes in a single one
df = pd.concat([pd.read_csv(f) for f in files if f.endswith(".csv")])
df = df.drop(columns=["dataset"])

In [5]:
models = df["method"].unique()
models = ["fmgp", "valla", "lla last_layer full", "lla last_layer kron", "ella"]
labels = ["FMGP", "VaLLA", "LLA* (full)", "LLA* (kron)", "ELLA"]
print("Models: ", models)

Models:  ['fmgp', 'valla', 'lla last_layer full', 'lla last_layer kron', 'ella']


In [6]:
def std(x):
    return np.std(x)

In [7]:
# Group by "method" and "resnet" and compute average and std
df = df.groupby(["method"]).agg(["mean", std])
df

RMSE            NLL                   Q-10  \
                          mean  std      mean           std     mean   
method                                                                 
ella                 38.902978  0.0  5.086775  5.119732e-07  0.13420   
fmgp                 38.902978  0.0  4.926038  3.220305e-14  0.09501   
lla last_layer full  38.902978  0.0  5.097362  4.586609e-09  0.12768   
lla last_layer kron  38.902978  0.0  5.097363  2.251736e-08  0.12768   
map                  38.902978  0.0  5.086787  0.000000e+00  0.13420   
valla                38.902978  0.0  4.934249  5.554702e-04  0.10573   

                                   Q-20               Q-30                ...  \
                          std      mean       std     mean           std  ...   
method                                                                    ...   
ella                 0.000000  0.275650  0.000000  0.42212  5.551115e-17  ...   
fmgp                 0.000000  0.195160  0.000000  0.30782  0.000000e+00  ...   
lla last_layer full  0.000000  0.261660  0.000000  0.40333  5.551115e-17  ...   
lla last_layer kron  0.000000  0.261660  0.000000  0.40333  5.551115e-17  ...   
map                  0.000000  0.275630  0.000000  0.42211  0.000000e+00  ...   
valla                0.000188  0.216882  0.000517  0.34556  7.915807e-04  ...   

                       train_time               test_time            seed  \
                             mean         std        mean        std mean   
method                                                                      
ella                  8885.148894  850.471009   80.939409   5.783420  2.0   
fmgp                   918.706687  126.237235    7.593059   0.972088  2.0   
lla last_layer full     29.377069    5.176054   10.745614   1.439321  2.0   
lla last_layer kron     51.721480    9.345935    9.875528   1.116760  2.0   
map                     60.436228    3.175907    5.431705   0.314976  2.0   
valla                27342.665522  190.641831  294.691218  22.942399  2.0   

                              sigma_noise               prior_precision  \
                          std        mean           std            mean   
method                                                                    
ella                 1.414214    1.000000  0.000000e+00        0.001000   
fmgp                 1.414214         NaN           NaN             NaN   
lla last_layer full  1.414214    0.954800  5.840039e-08        6.364602   
lla last_layer kron  1.414214    0.954797  2.384186e-08        7.741396   
map                  1.414214    1.000000  0.000000e+00             NaN   
valla                1.414214         NaN           NaN             NaN   

                                   
                              std  
method                             
ella                 0.000000e+00  
fmgp                          NaN  
lla last_layer full  8.643257e-06  
lla last_layer kron  7.629395e-07  
map                           NaN  
valla                         NaN  

[6 rows x 36 columns]

In [8]:
fig, axis = plt.subplots(1, 3, figsize=(20, 5))
# Plot NLL in axis 0
for j, model in enumerate(models):
    nll = df.loc[model, ("NLL", "mean")]
    std = df.loc[model, ("NLL", "std")]
    axis[0].bar(j, nll, yerr=std, label=labels[j], color=f"C{j}")
    axis[0].set_title("NLL")
    axis[0].set_xticks(range(len(models)))
    axis[0].set_xticklabels(labels)

    # Plot CRPS
    crps = df.loc[model, ("CRPS", "mean")]
    std = df.loc[model, ("CRPS", "std")]
    axis[1].bar(j, crps, yerr=std, label=labels[j], color=f"C{j}")
    axis[1].set_title("CRPS")
    axis[1].set_xticks(range(len(models)))
    axis[1].set_xticklabels(labels)

    # Plot Q
    q = df.loc[model, ("CQM", "mean")]
    std = df.loc[model, ("CQM", "std")]
    axis[2].bar(j, q, yerr=std, label=labels[j], color=f"C{j}")
    axis[2].set_title("CQM")
    axis[2].set_xticks(range(len(models)))
    axis[2].set_xticklabels(labels)

Error in callback <function _draw_all_if_interactive at 0x15108d0bfc40> (for post_execute), with arguments args (),kwargs {}:


RuntimeError: Failed to process string with tex because latex could not be found

RuntimeError: Failed to process string with tex because latex could not be found

<Figure size 2000x500 with 3 Axes>

In [9]:
# Get Q-10, Q-20.. columns in array
for i, model in enumerate(models):
    Q = []
    Q_std = []
    for q in range(10, 100, 10):
        q = f"Q-{q}"
        q_values = df.loc[model, (q, "mean")]
        Q.append(q_values)
        q_std = df.loc[model, (q, "std")]
        Q_std.append(q_std)
    plt.plot(np.arange(0, 1.1, 0.1), [0, *Q, 1], label=labels[i])
    plt.fill_between(
        np.arange(0, 1.1, 0.1),
        [0, *Q, 1],
        [0, *(np.array(Q) - np.array(Q_std)), 1],
        alpha=0.2,
    )
plt.plot(
    np.arange(0, 1.1, 0.1),
    np.arange(0, 1.1, 0.1),
    label="Perfect calibration",
    linestyle="--",
    color="black",
)
plt.legend()

Error in callback <function _draw_all_if_interactive at 0x15108d0bfc40> (for post_execute), with arguments args (),kwargs {}:


RuntimeError: Failed to process string with tex because latex could not be found

RuntimeError: Failed to process string with tex because latex could not be found

<Figure size 1600x900 with 1 Axes>

In [10]:
# Get Q-10, Q-20.. columns in array
sep = 0.015
er = np.arange(-2, 3, 1) * sep
for i, model in enumerate(models):
    Q = []
    Q_std = []
    for q in range(10, 100, 10):
        q = f"Q-{q}"
        q_values = df.loc[model, (q, "mean")]
        Q.append(q_values)
        q_std = df.loc[model, (q, "std")]
        Q_std.append(q_std)
    b = plt.bar(np.arange(0, 1.1, 0.1) + er[i], [0, *Q, 1], width=sep, label=labels[i])
plt.axhline(0.0, 0.04, 0.12, linestyle="--", color="black")
plt.axhline(0.1, 0.12, 0.20, linestyle="--", color="black")
plt.axhline(0.2, 0.2, 0.28, linestyle="--", color="black")
plt.axhline(0.3, 0.29, 0.37, linestyle="--", color="black")
plt.axhline(0.4, 0.38, 0.45, linestyle="--", color="black")
plt.axhline(0.5, 0.46, 0.54, linestyle="--", color="black")
plt.axhline(0.6, 0.55, 0.62, linestyle="--", color="black")
plt.axhline(0.7, 0.63, 0.71, linestyle="--", color="black")
plt.axhline(0.8, 0.72, 0.79, linestyle="--", color="black")
plt.axhline(0.9, 0.80, 0.88, linestyle="--", color="black")
plt.axhline(1.0, 0.88, 0.96, linestyle="--", color="black")

plt.legend()

Error in callback <function _draw_all_if_interactive at 0x15108d0bfc40> (for post_execute), with arguments args (),kwargs {}:


RuntimeError: Failed to process string with tex because latex could not be found

RuntimeError: Failed to process string with tex because latex could not be found

<Figure size 1600x900 with 1 Axes>

In [11]:
# Concatenate all dataframes in a single one
nll = []
crps = []
Q = []
for i in range(5):
    nll.append(np.loadtxt(f"ella_{i}_nll.txt"))
    crps.append(np.loadtxt(f"ella_{i}_crps.txt"))
    Q.append(np.loadtxt(f"ella_{i}_cqm.txt"))
nll = np.array(nll)
crps = np.array(crps)
Q = np.array(Q)

nll_mean = np.mean(nll, axis=0)
nll_std = np.std(nll, axis=0) / np.sqrt(5)
crps_mean = np.mean(crps, axis=0)
crps_std = np.std(crps, axis=0) / np.sqrt(5)
Q_mean = np.mean(Q, axis=0)
Q_std = np.std(Q, axis=0) / np.sqrt(5)


In [12]:
evaluations = np.loadtxt("ella_0_evaluations.txt")


In [13]:
fig, axis = plt.subplots(1, 3, figsize=(20, 5))
axis[0].plot(evaluations[1:], nll_mean[1:])
axis[0].set_title("NLL")

axis[1].plot(evaluations[1:], crps_mean[1:])
axis[1].set_title("CRPS")

axis[2].plot(evaluations[1:], Q_mean[1:])
axis[2].set_title("Q")

plt.show()


RuntimeError: Failed to process string with tex because latex could not be found

<Figure size 2000x500 with 3 Axes>